In [1]:
from kiara import KiaraAPI
import os
import networkx
from networkx.readwrite import json_graph

Process based on kiara version 0.4.21

In [2]:
data_folder = "/home/markus/projects/kiara/playground/anom/"

personnel_csv = os.path.join(data_folder, "personnel.csv")
coords_csv = os.path.join(data_folder, 'anommedicalgeocoded-edited.csv')

api = KiaraAPI.instance()

In [3]:
# convert kiara table to pandas for previewing
def kiara_to_pandas(table_value):
    table_obj = table_value.data
    arrow_table = table_obj.arrow_table
    df = arrow_table.to_pandas()
    return df

### 1. CSV file onboarding

In [4]:
! kiara operation explain import.table.from.csv_file


╭─ Operation: import.table.from.csv_file ──────────────────────────────────────╮
│                                                                              │
│   Documentation   Import a table from a csv file.                            │
│                                                                              │
│   Inputs                                                                     │
│                     field                                                    │
│                     name        type     descrip…   Required   Default       │
│                    ──────────────────────────────────────────────────────    │
│                     path        string   The        yes        -- no         │
│                                          local                 default       │
│                                          path to               --            │
│                                          the                                 │
│                          

In [6]:
onboarding_result = api.run_job(operation="import.table.from.csv_file", inputs={'path':personnel_csv})

In [7]:
# preview data
table_value = api.get_value(onboarding_result['table'].value_id)
df = kiara_to_pandas(table_value)

In [8]:
df.head()

,archref,name,placesdisplay,placesnormalized,corp,startdate,enddate,daterange,alldates,fullLink,fullRecord
0,"<a class=""archref"" href=""/ark:/61561/up424lfky...",Abadie,['Sainte-Lucie'],"['Sainte-Lucie, Île (Antilles)']",,1787.0,1787.0,1787-1787,1787,/ark:/61561/up424lfky,"Abadie, huissier à Sainte-Lucie 1787"
1,"<a class=""archref"" href=""/ark:/61561/up424tntx...","Abbadie, d'",['Canada'],"['Canada, Colonie française']",Régiment de Carignan-Salières,1665.0,1708.0,1665-1708,"1665, 1708",/ark:/61561/up424tntx,"Abbadie, d', enseigne dans le régiment de Cari..."
2,"<a class=""archref"" href=""/ark:/61561/up424icjr...","Abeille, Jean",['Port-au-Prince'],"['Port-au-Prince (Saint-Domingue, Île de)']",,1788.0,1789.0,1788-1789,"1788, 1789",/ark:/61561/up424icjr,"Abeille, Jean, lieutenant de milices du batail..."
3,"<a class=""archref"" href=""/ark:/61561/up424tnvz...","Abeille, Jean Joseph",['Pondichéry'],['Pondichéry (Inde)'],Conseil supérieur (Pondichéry ; Inde),1769.0,1772.0,1769-1772,"1769, 1772",/ark:/61561/up424tnvz,"Abeille, Jean Joseph, conseiller au Conseil su..."
4,"<a class=""archref"" href=""/ark:/61561/up424ezxy...","Abeille, Pierre",['Port-au-Prince'],"['Port-au-Prince (Saint-Domingue, Île de)']",,1774.0,1787.0,1774-1787,"1774, 1787",/ark:/61561/up424ezxy,"Abeille, Pierre, négociant à Port-au-Prince à ..."


In [9]:
df.tail()

,archref,name,placesdisplay,placesnormalized,corp,startdate,enddate,daterange,alldates,fullLink,fullRecord
19333,"<a class=""archref"" href=""/ark:/61561/up424tnvp...","Zemard, Ambroise",['île Royale'],"['Royale, Île (Canada)']",,1741.0,1741.0,1741-1741,1741,/ark:/61561/up424tnvpsny,"Zemard, Ambroise, habitant de l'île Royale, co..."
19334,"<a class=""archref"" href=""/ark:/61561/up424nhpj...","Zévallos, de",['Sainte-Anne'],"['Sainte-Anne (Guadeloupe, Île de la)']",,1770.0,1770.0,1770-1770,1770,/ark:/61561/up424nhpjmip,"Zévallos, de, commandant des milices du quarti..."
19335,"<a class=""archref"" href=""/ark:/61561/up424tnvp...","Zévallos, Edouard de",,,Régiment de la Guadeloupe,1786.0,1791.0,1786-1791,"1786, 1790, 1791",/ark:/61561/up424tnvpspa,"Zévallos, Edouard de, sous-lieutenant au régim..."
19336,"<a class=""archref"" href=""/ark:/61561/up424wqys...",Zunbergen,['Guyane'],['Guyane française'],,1784.0,1784.0,1784-1784,1784,/ark:/61561/up424wqysvtt,"Zunbergen, accusé de malversations en Guyane 1784"
19337,"<a class=""archref"" href=""/ark:/61561/up424vpxr...","Zweerts, Constantin Christian",['Tabago'],"['Tabago, Île (Antilles)']",,1782.0,1787.0,1782-1787,"1782, 1787",/ark:/61561/up424vpxruto,"Zweerts, Constantin Christian, interprète de l..."


In [10]:
# checking if missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19338 entries, 0 to 19337
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   archref           19338 non-null  object 
 1   name              19338 non-null  object 
 2   placesdisplay     19338 non-null  object 
 3   placesnormalized  19338 non-null  object 
 4   corp              19338 non-null  object 
 5   startdate         19251 non-null  float64
 6   enddate           19251 non-null  float64
 7   daterange         19338 non-null  object 
 8   alldates          19338 non-null  object 
 9   fullLink          19338 non-null  object 
 10  fullRecord        19338 non-null  object 
dtypes: float64(2), object(9)
memory usage: 1.6+ MB


### 2. Columns rename

In [11]:
! kiara operation explain anom_processing.column_names_replace


╭─ Operation: anom_processing.column_names_replace ────────────────────────────╮
│                                                                              │
│   Documentation   Replace columns names.                                     │
│                                                                              │
│   Inputs                                                                     │
│                     field                                                    │
│                     name        type    descript…   Required   Default       │
│                    ──────────────────────────────────────────────────────    │
│                     table       table   The table   yes        -- no         │
│                                         for which              default       │
│                                         one or                 --            │
│                                         more                                 │
│                          

In [12]:
col_map = column_map = {
    "name": "foaf:name",
    "placesdisplay": "gn:name",
    "daterange": "dc:date",
    "fullLink": "dc:identifier"
}

In [13]:
rename_columns_result = api.run_job(operation="anom_processing.column_names_replace", inputs={'table':table_value, 'columns_map':col_map})

In [14]:
rename_columns_result

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field   value                                                                                                                          │
│  ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────  │
│   table                                                                                                                                  │
│             archref    foaf:nam   gn:name    placesno   corp        startdat   enddate   dc:date     alldates   dc:identi   fullReco     │
│            ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────    │
│             <a class   Abadie     ['Sainte   ['Sainte               1787.0     1787.0    1787-1787   1787       /ark:/615   Abadie,      │
│             <a class   Abbadie,   ['Canada   ['Canada   Régiment    1665.0     1708.0    1665-1708   1665, 17   /ark:/615   Abbadie,     │
│             <a class   Abeille,   ['Port-a   ['Port-a               1788.0     1789.0    1788-1789   1788, 17   /ark:/615   Abeille,     │
│             <a class   Abeille,   ['Pondic   ['Pondic   Conseil s   1769.0     1772.0    1769-1772   1769, 17   /ark:/615   Abeille,     │
│             <a class   Abeille,   ['Port-a   ['Port-a               1774.0     1787.0    1774-1787   1774, 17   /ark:/615   Abeille,     │
│             <a class   Abel, Je   ['Guadel   ['Guadel               1784.0     1784.0    1784-1784   1784       /ark:/615   Abel, Je     │
│             <a class   Abel, Ol   ['Canada   ['Canada               1732.0     1749.0    1732-1749   1732, 17   /ark:/615   Abel, Ol     │
│             <a class   Ablé, Pi                                     1777.0     1777.0    1777-1777   1777       /ark:/615   Ablé, Pi     │
│             <a class   Abon, Ja                         Régiment    1773.0     1779.0    1773-1779   1773, 17   /ark:/615   Abon, Ja     │
│             <a class   Accaron,                                     1735.0     1766.0    1735-1766   1735, 17   /ark:/615   Accaron,     │
│             <a class   Accolans   ['Saint-   ['Saint-               1767.0     1770.0    1767-1770   1767, 17   /ark:/615   Accolans     │
│             <a class   Achard,    ['Pointe   ['Pointe               1787.0     1787.0    1787-1787   1787       /ark:/615   Achard,      │
│             <a class   Achard,    ['Basse-   ['Basse-               1787.0     1787.0    1787-1787   1787       /ark:/615   Achard,      │
│             <a class   Achard d   ['Cap']    ['Le Cap   Conseil s   1769.0     1776.0    1769-1776   1769, 17   /ark:/615   Achard d     │
│             <a class   Achard d   ["Saint-   ["Saint-               1734.0     1779.0    1734-1779   1734, 17   /ark:/615   Achard d     │
│             <a class   Aché       ['Embrun   ['Embrun   Régiment    1788.0     1788.0    1788-1788   1788       /ark:/615   Aché, co     │
│             ...        ...        ...        ...        ...         ...        ...       ...         ...        ...         ...          │
│             ...        ...        ...        ...        ...         ...        ...       ...         ...        ...         ...          │
│             <a class   Yvon, Gi   ['Saint-   ['Saint-   Régiment    1775.0     1784.0    1775-1784   1775, 17   /ark:/615   Yvon, Gi     │
│             <a class   Yvon, Jo   ['Sainte   ['Sainte               1763.0     1763.0    1763-1763   1763       /ark:/615   Yvon, Jo     │
│             <a class   Yvon, Lo   ['île de   ['France               1771.0     1790.0    1771-1790   1771, 17   /ark:/615   Yvon, Lo     │
│             <a class   Yvon       ['île de   ['Fra

In [15]:
rename_columns_value = api.get_value(rename_columns_result['table'].value_id)

### 3. Check if all location data available in dataset and all coordinates present in coordinates data

If the objective is to be able to display the dataset on a map, all rows must contain a normalized location name, and
each location name needs to have a latitude and a longitude

#### 3.1 Remove rows that contain nans in location place

In [16]:
with_loc_result = api.run_job(operation="anom_processing.remove_nans", inputs={'table':rename_columns_value, 'column':'placesnormalized'})

At the moment the module doesn't output the list of rows containing nans in normalized location name col

In [17]:
with_loc_value = api.get_value(with_loc_result['table'].value_id)

#### 3.1 Access coordinates CSV

In [19]:
coords_onboarding_result = api.run_job(operation="import.table.from.csv_file", inputs={'path': coords_csv})

In [20]:
coords_table_value = api.get_value(coords_onboarding_result['table'].value_id)

#### 3.2 Rename latitude and longitude

I'm temporary leaving this step out for now since there are 2 place names, I'm wondering if we shouldn't keep the standardised one for the normalized place name col as it is common accross tables.

#### 3.2 Check if all coordinates present

In [21]:
! kiara operation explain anom_processing.coords_check


╭─ Operation: anom_processing.coords_check ────────────────────────────────────╮
│                                                                              │
│   Documentation   This module aims at comparing two tables: one table that   │
│                   provides observations with a place name, and one table     │
│                   that                                                       │
│                                                                              │
│                   includes latitudes, longitudes and place names.            │
│                   A verification is performed to see if all place names of   │
│                   the first dataset are included in the second dataset.      │
│                   In this specific example, the first table includes         │
│                   several place names per row.                               │
│                   At the moment the module only covers the case of the       │
│                   specifi

In [22]:
coords_check_result = api.run_job(operation="anom_processing.coords_check", inputs={'table1':with_loc_value, 'table2':coords_table_value, "column1":'placesnormalized',"column2":'normalized'})

In [23]:
coords_check_result['result'].data

'Missing coordinates.'

In [24]:
# uncomment to see locations that are not present in the geocoded places file
# coords_check_result['places_list'].data

### 4. Filter out some columns

In [25]:
! kiara operation explain table_filter.select_columns


╭─ Operation: table_filter.select_columns ─────────────────────────────────────╮
│                                                                              │
│   Documentation   -- n/a --                                                  │
│                                                                              │
│   Inputs                                                                     │
│                     field                                                    │
│                     name       type      descrip…   Required   Default       │
│                    ──────────────────────────────────────────────────────    │
│                     value      table     A value    yes        -- no         │
│                                          of type               default       │
│                                          'table'.              --            │
│                     columns    list      The name   no         -- no         │
│                          

In [26]:
to_include = ['foaf:name','dc:date','dc:identifier']

In [27]:
filter_columns_result = api.run_job(operation="table_filter.select_columns", inputs={'value':rename_columns_value, 'columns':to_include})

In [28]:
filter_columns_result

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                                          │
│   field   value                                                                                                                          │
│  ──────────────────────────────────────────────────────────────────────────────────────────                                              │
│   value                                                                                                                                  │
│             foaf:name                               dc:date     dc:identifier                                                            │
│            ──────────────────────────────────────────────────────────────────────────────                                                │
│             Abadie                                  1787-1787   /ark:/61561/up424lfky                                                    │
│             Abbadie, d'                             1665-1708   /ark:/61561/up424tntx                                                    │
│             Abeille, Jean                           1788-1789   /ark:/61561/up424icjr                                                    │
│             Abeille, Jean Joseph                    1769-1772   /ark:/61561/up424tnvz                                                    │
│             Abeille, Pierre                         1774-1787   /ark:/61561/up424ezxy                                                    │
│             Abel, Jean                              1784-1784   /ark:/61561/up424lgfu                                                    │
│             Abel, Ollivier                          1732-1749   /ark:/61561/up424dyyx                                                    │
│             Ablé, Pierre                            1777-1777   /ark:/61561/up424mhiz                                                    │
│             Abon, Jacques Auguste d'                1773-1779   /ark:/61561/up424idfo                                                    │
│             Accaron, Jean Augustin                  1735-1766   /ark:/61561/up424mhkb                                                    │
│             Accolans de Colonge, Jean Baptiste d'   1767-1770   /ark:/61561/up424dy2e                                                    │
│             Achard, Antoine                         1787-1787   /ark:/61561/up424xsxk                                                    │
│             Achard, Antoine                         1787-1787   /ark:/61561/up424jekv                                                    │
│             Achard de Champroger, Louis Charles     1769-1776   /ark:/61561/up424kfmz                                                    │
│             Achard de La Lente, André Thomas        1734-1779   /ark:/61561/up424kgdr                                                    │
│             Aché                                    1788-1788   /ark:/61561/up424qmkk                                                    │
│             ...                                     ...         ...                                                                      │
│             ...                                     ...         ...                                                                      │
│             Yvon, Gilles Joseph                     1775-1784   /ark:/61561/up424mgoikfg                                                 │
│             Yvon, Joseph Emmanuel                   1763-1763   /ark:/61561/up424dx5z1xl                                                 │
│             Yvon, Louis Christophe Henri            1771-1790   /ark:/61561/up424oiqkmju                                                 │
│             Yvon                                  

In [29]:
# preview data
table_value = api.get_value(filter_columns_result['value'].value_id)
df = kiara_to_pandas(table_value)

In [30]:
df.head()

,foaf:name,dc:date,dc:identifier
0,Abadie,1787-1787,/ark:/61561/up424lfky
1,"Abbadie, d'",1665-1708,/ark:/61561/up424tntx
2,"Abeille, Jean",1788-1789,/ark:/61561/up424icjr
3,"Abeille, Jean Joseph",1769-1772,/ark:/61561/up424tnvz
4,"Abeille, Pierre",1774-1787,/ark:/61561/up424ezxy


#### 5. Save table in data registry

In [31]:
table_alias = "20221103_test"

In [32]:
api.store_value(table_value, table_alias)

StoreValueResult(value=Value(id=2721ab0f-2dad-4a01-9bb9-b538fd3f8ecc, type=table, status=set, initialized=True optional=False), aliases=['20221103_test'], persisted_data=PersistedData(model_id=zdpuAu5iFaJ6x2GQ7UZe6rQ7psUsFKcJAkS3FrwMzGdJwVX62, category=instance.persisted_data, fields=[data_type, data_type_config, serialization_profile, metadata, hash_codec, archive_id, chunk_id_map]), error=None)

In [33]:
# chack data registry
! kiara data list


╭─ Available aliases ──────────────────────────────────────────────────────────╮
│                                                                              │
│   alias           type       size                                            │
│  ─────────────────────────────────                                           │
│   20221103_test   table   1.21 MB                                            │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯
